# Introduction / Business Problem

Chicago has been declining in population for the past 3 years. It's rare for a metropolitan area to have a net decrease in population year-over-year. Here is more information about the decline: https://www.chicagotribune.com/news/local/breaking/ct-met-chicago-cook-county-population-20180320-story.html  
  
The types of venues that are present in a given zip code, among other types of information available from foursquare, may serve as good predictors of whether that zip code is likely to experience population decline. My goal for this project is to make a classifier for a given zip code to predict if it will experience decline or not. This kind of analysis could be helpful to demographers, sociologists, city planners, politicians, economists, neighborhood associations, and probably a few more professions.

# Data

On the data, I plan to get the venues from the explore API from foursquare for each zip code in Chicago. I will use this data for features. Then I plan to use the total population for each of the zip codes from the US Census Bureau American Fact Finder tool to determine if there was decline between the years 2016 and 2017.  
  
Here is a list of all chicago zip codes: http://www.city-data.com/zipmaps/Chicago-Illinois.html  
Here is the link to the census bureau data that I will import as a two CSV files, one for 2016 and one for 2017: https://factfinder.census.gov/bkmk/table/1.0/en/ACS/17_5YR/B01003/8600000US60007|8600000US60018|8600000US60068|8600000US60106|8600000US60131|8600000US60176|8600000US60601|8600000US60602|8600000US60603|8600000US60604|8600000US60605|8600000US60606|8600000US60607|8600000US60608|8600000US60609|8600000US60610|8600000US60611|8600000US60612|8600000US60613|8600000US60614|8600000US60615|8600000US60616|8600000US60617|8600000US60618|8600000US60619|8600000US60620|8600000US60621|8600000US60622|8600000US60623|8600000US60624|8600000US60625|8600000US60626|8600000US60628|8600000US60629|8600000US60630|8600000US60631|8600000US60632|8600000US60633|8600000US60634|8600000US60636|8600000US60637|8600000US60638|8600000US60639|8600000US60640|8600000US60641|8600000US60642|8600000US60643|8600000US60644|8600000US60645|8600000US60646|8600000US60647|8600000US60649|8600000US60651|8600000US60652|8600000US60653|8600000US60654|8600000US60655|8600000US60656|8600000US60657|8600000US60659|8600000US60660|8600000US60661|8600000US60706|8600000US60707|8600000US60714|8600000US60804|8600000US60827

Use autogenerated code to import the csv files from IBM Cloud Object Storage. df_data_1 is for 2016. df_data_2 is for 2017.

In [6]:
# The code was removed by Watson Studio for sharing.

,GEO.id,GEO.id2,GEO.display-label,Estimate; Total,Margin of Error; Total
0,8600000US60007,60007,ZCTA5 60007,33733,161
1,8600000US60018,60018,ZCTA5 60018,30519,816
2,8600000US60068,60068,ZCTA5 60068,37567,323
3,8600000US60106,60106,ZCTA5 60106,20215,328
4,8600000US60131,60131,ZCTA5 60131,18072,168


In [7]:
body = client_0830bd8539294ecb8e07bb128b7fed36.get_object(Bucket='capstonesegmentingandsplitting-donotdelete-pr-uzmj8rdmck50ke',Key='ACS_17_5YR_B01003.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_2 = pd.read_csv(body)
df_data_2.head()

,GEO.id,GEO.id2,GEO.display-label,Estimate; Total,Margin of Error; Total
0,8600000US60007,60007,ZCTA5 60007,33691,191
1,8600000US60018,60018,ZCTA5 60018,30955,996
2,8600000US60068,60068,ZCTA5 60068,37810,312
3,8600000US60106,60106,ZCTA5 60106,20300,342
4,8600000US60131,60131,ZCTA5 60131,18070,129


Clean up the data into one dataframe.

In [29]:
population_16 = df_data_1[['GEO.id2','Estimate; Total']]
population_16.columns = ['Zip Code', 'Population 2016']
population_16.head()

,Zip Code,Population 2016
0,60007,33733
1,60018,30519
2,60068,37567
3,60106,20215
4,60131,18072


In [30]:
population_17 = df_data_2[['GEO.id2','Estimate; Total']]
population_17.columns = ['Zip Code','Population 2017']
population_17.head()

,Zip Code,Population 2017
0,60007,33691
1,60018,30955
2,60068,37810
3,60106,20300
4,60131,18070


In [31]:
population = population_16
population['Population 2017'] = population_17['Population 2017']
population.head()

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,Zip Code,Population 2016,Population 2017
0,60007,33733,33691
1,60018,30519,30955
2,60068,37567,37810
3,60106,20215,20300
4,60131,18072,18070


In [34]:
population['Decline'] = population['Population 2016'] - population['Population 2017'] > 0
population.head()

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Zip Code,Population 2016,Population 2017,Decline
0,60007,33733,33691,True
1,60018,30519,30955,False
2,60068,37567,37810,False
3,60106,20215,20300,False
4,60131,18072,18070,True


Here is an example of the explore API from foursquare:

In [42]:
import requests

CLIENT_ID = 'GC0PBW0SRCSK1CRI3F4DJVH0MMFHZX3R1MXJCWDHJLD4TACG' # your Foursquare ID
CLIENT_SECRET = 'HGJ5IRVIOMNM0OCAURIDLFJJBKWJW4LCKCHNRRJ21Q3KWSWL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
radius = 2000
LIMIT = 100
test_lat= 40.1038
test_long= -83.0200
name = 'Worthington'

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            test_lat, 
            test_long, 
            radius, 
            LIMIT)

# make the GET request
results = requests.get(url).json()["response"]['groups'][0]['items']
venues_list=[]
# return only relevant information for each nearby venue
venues_list.append([(
            name, 
            test_lat, 
            test_long, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
nearby_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Worthington,40.1038,-83.02,"Pies & Pints - Worthington, OH",40.109009,-83.018817,Pizza Place
1,Worthington,40.1038,-83.02,J Liu Restaurant & Bar,40.104396,-83.016468,Restaurant
2,Worthington,40.1038,-83.02,Amano's,40.109415,-83.019863,Pizza Place
3,Worthington,40.1038,-83.02,Worthington Winter Farmers Market,40.108385,-83.019309,Market
4,Worthington,40.1038,-83.02,Blaze Pizza,40.107374,-83.015947,Pizza Place
